# 4. GLMのモデル選択

Q. モデル選択とは

A. 
* 良い予測をするモデルを探すこと
* 「当てはまりのよさ」だけで選んではいけない
    * 複雑な統計モデルほど、観測データへの当てはまりは良くなる

Q. AICとは？

A.
* 「良い予測をするモデルが良いモデルである」という考えに基づいて設計された、モデル選択基準
* 「当てはまりの良さ重視」とは異なる考え方

## 4.1. データはひとつ、モデルはたくさん

* 第3章では、同じデータについて、いくつかの統計モデルで説明した
* 統計モデルのパラメータを最尤推定する際には、対数尤度が「いま手元にある観測データへの当てはまりの良さ」であると考え、これを最大にするようなパラメータの値を探した
* では、最大対数尤度こそがモデルの良さであると考えれば良いのだろうか？
* ->そうではない、というのが本章の要点

## 4.2. 統計モデルのあてはまりの悪さ：逸脱度

Q. 逸脱度とは？

A.
* あてはまりの良さである最大対数尤度を変形した統計量

---

Q. 

* ある対数尤度$ \log L( \{ \beta_j \} )$を$\log L$ と表記する
* この$\log L$を最大にするパラメータを探すのが最尤推定法
* 最大対数尤度を $\log L^*$と表記する


とした時、逸脱度の定義式は？

A. 
$$
D = -2\log L^*
$$
* 「あてはまりの良さ」ではなく「あてはまりの悪さ」を表現する指標
* 数値が大きいほど、あてはまりが悪い
* 最大対数尤度$\log L^*$に$-2$をかけている

---

Q. 残差逸脱度とは？

A.
* $D -$(ポアソン分布モデルで可能な最小逸脱度）
* この最小逸脱度をフルモデルと呼び、データ数と同じだけのパラメータを使って当てはめたモデル
* つまり残差逸脱度は、もっともあてはまりの良いフルモデルを基準とする「あてはまりの悪さ」の相対値。
* 残差逸脱度が最大になるのは、「もっともあてはまりの悪いモデル」の場合
* ポアソン分布だとこれは、もっともパラメーター数の少ないモデル null modelである。

---

### 体サイズ$x_i$だけに依存するモデルで計算する

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as scs
%matplotlib inline

In [2]:
data = pd.read_csv("data3a.csv")

In [3]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

/Users/takizawa/dev/python/books/midori/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [4]:
model = smf.glm('y ~ x',data=data,family=sm.families.Poisson())

In [5]:
result = model.fit()

In [6]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                  100
Model:                            GLM   Df Residuals:                       98
Model Family:                 Poisson   Df Model:                            1
Link Function:                    log   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:                -235.39
Date:                Thu, 06 Jul 2017   Deviance:                       84.993
Time:                        00:48:26   Pearson chi2:                     83.8
No. Iterations:                     4                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.2917      0.364      3.552      0.000       0.579       2.005
x              0.0757      0.036      2.125      0.034       0.006       0.145
==============================================================================
"""

* Pythonだと、Deviance 84.993と表示されている。おそらくResidual Deviance
* Null Devianceは表示されない

### full modelの逸脱度の計算

* 100個のdata.yに対し、100個のパラメータを用いて当てはめることを考える。
* ポアソン回帰で可能な他のどのモデルよりも「当てはまりの良さ」である対数尤度は大きくなる。
* ある1つのデータに対し、そのデータの値を平均とした、ポアソン分布の（最大）対数尤度を取得するlambda式 dpois を定義。
* data.yの1つずつに対し、dpoisを当てはめ計算。
* それらを足し合わせてfull modelの最大対数尤度を計算。
* $-2$を掛け算し、最小逸脱度を求める。

In [7]:
dpois = lambda x:scs.poisson.logpmf(x, x)

In [8]:
data.y

0      6
1      6
2      6
3     12
4     10
5      4
6      9
7      9
8      9
9     11
10     6
11    10
12     6
13    10
14    11
15     8
16     3
17     8
18     5
19     5
20     4
21    11
22     5
23    10
24     6
25     6
26     7
27     9
28     3
29    10
      ..
70    10
71     8
72     8
73     7
74     5
75     6
76     8
77     9
78     9
79     6
80     7
81    10
82     6
83    11
84    11
85    11
86     5
87     6
88     4
89     5
90     6
91     5
92     8
93     5
94     9
95     8
96     6
97     8
98     7
99     9
Name: y, Length: 100, dtype: int64

In [9]:
dpois(data.y)

array([-1.8286944 , -1.8286944 , -1.8286944 , -2.1683347 , -2.07856164,
       -1.63287639, -2.02680628, -2.02680628, -2.02680628, -2.12545985,
       -1.8286944 , -2.07856164, -1.8286944 , -2.07856164, -2.12545985,
       -1.96907057, -1.4959226 , -1.96907057, -1.74030218, -1.74030218,
       -1.63287639, -2.12545985, -1.74030218, -2.07856164, -1.8286944 ,
       -1.8286944 , -1.90379032, -2.02680628, -1.4959226 , -2.07856164,
       -1.30685282, -2.02680628, -2.07856164, -1.74030218, -2.12545985,
       -2.07856164, -1.63287639, -1.96907057, -2.02680628, -2.1683347 ,
       -1.96907057, -2.02680628, -1.96907057, -1.8286944 , -1.8286944 ,
       -2.07856164, -2.07856164, -2.02680628, -2.1683347 , -1.8286944 ,
       -2.24441857, -1.8286944 , -1.90379032, -2.02680628, -1.8286944 ,
       -1.90379032, -2.02680628, -2.20782221, -2.02680628, -2.20782221,
       -1.90379032, -1.96907057, -2.07856164, -1.90379032, -2.1683347 ,
       -1.8286944 , -2.27851837, -1.4959226 , -1.63287639, -1.82

In [10]:
max_logl = sum(dpois(data.y))
max_logl

-192.8897525244958

In [11]:
min_dev = sum(dpois(data.y)) * -2
min_dev

385.77950504899161

* 最小逸脱度は385程度となった

### null modelの作成

* ポアソン回帰ではもっともパラメータ数の少ないモデル。切片 $\beta_1$だけのモデル（パラメータ数$k=1$）
* 一定モデルとも呼ばれる。

In [12]:
model = smf.glm('y ~ 1',data=data,family=sm.families.Poisson())

In [13]:
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                  100
Model:                            GLM   Df Residuals:                       99
Model Family:                 Poisson   Df Model:                            0
Link Function:                    log   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:                -237.64
Date:                Thu, 06 Jul 2017   Deviance:                       89.507
Time:                        00:48:26   Pearson chi2:                     87.1
No. Iterations:                     4                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.0580      0.036     57.586      0.000       1.988       2.128
==============================================================================
"""

* Devianceは89.507とResidualDevianceと一致している
* 一定モデルの最大対数尤度は-237.64となっており、逸脱度は475程度、最小逸脱度との差は475-385=90となり、Devianceと一致

### (復習)一定モデルなら $\lambda$ はデータの平均と一致
* $\log \lambda = \beta_1$

In [14]:
data.y.mean()

7.8300000000000001

In [15]:
np.log(data.y.mean())

2.0579625100027119

## 4.3. モデル選択基準 AIC

* パラメータ数の多い統計モデルほど、データへのあてはまりが良くなる
* しかしそれは「たまたま得られたデータへの当てはめ向上を目的とする特殊化」であり、統計モデルの「予測の良さ」を損なっているのかもしれない

Q. モデル選択とは？

A.
* 複数の統計モデルの中から、何らかの基準で良いモデルを選択すること
* よく使われているモデル選択基準の1つがAIC(Akaike's information criterion)

Q. AICとは？

A.
* 予測の良さを重視するモデル選択基準
* $AIC = -2 \{ $ ( 最大対数尤度 ) - (最尤推定したパラメータ数)$\}$ 
$$= -2(\log L^* - k) $$
$$= D + 2k$$
* このAICが一番小さいモデルが良いモデルとなる

## 4.4. AICを説明するためのまた別の問題

* AICがなぜ統計モデルの予測力を表しているのか考える。

Q. ネストしているモデルとは？

A.

* 一定モデル($\log \lambda_i = \beta_1$) と xモデル（$\log \lambda_i = \beta_1 + \beta_2 x_i$）のように、一方のモデルに他方が含まれているようなくみあわせ。（$\beta_2 = 0$とすれば、xモデルは一定モデルと同様になる）

## 4.5. なぜAICでモデル選択して良いのか？

### 4.5.1. 統計モデルの予測の良さ：平均対数尤度

* 平均対数尤度$E(\log L)$は統計モデルの予測の良さを表す量
* 真の統計モデルに基づくデータセットに対し、すでに推定されたモデルの当てはまりの良さを対数尤度で評価し、その平均をとったもの

### 統計モデルのパラメータ推定の概念
* 人間には見えない真の統計モデルがある
* それを元に、観測データがサンプリングされる
* 観測データを元にパラメータ推定する
* そのため、パラメータ推定によって求められたパラメータと最大対数尤度は、たまたま得られた観測データ自身に対する当てはまりの良さであり、真の統計モデルに似ているかは考慮されない


Q. データ解析の狙いは？

A. 
* 観測される現象の背後にある「仕組み」の特定、もしくはそれを近似的に代替しうる統計モデルの構築
* そのため、「たまたま得られた」観測データへの当てはまりの良さを追求してはならない

Q. 推定された統計モデルが真の統計モデルにどれぐらい近いのかを調べる方法は？

A. 
* 真の統計モデルがわかっているのなら、そこから「予測の良さ評価用のデータ」を生成し、推定された一定モデルのどの程度当てはまるのか調べる
* 教科書の喩えだと、50個のサンプルを200セット作成。
* 各セットに対して対数尤度で評価する。その200の結果の平均を、平均対数尤度と呼ぶ。
* しかし、真の統計モデルがわかってることなどほぼない（わかってたら観測しない）

### 4.5.2. 最大対数尤度のバイアス補正
* たまたま得られた観測データで推定されたモデルでは、当てはまりの良さが過大評価される（常に過大評価される訳ではない）
* 平均対数尤度の作成を12回繰り返す。すると、最大対数尤度（観測データへの当てはまり）と平均対数尤度（真の統計モデルから生成したデータへの当てはまり）は必ずしもどちらが良いとは言えないことがわかる

Q. バイアスとは？

A. 
* $ b = \log L^* - E(\log L) $
* 「推定用データ生成、パラメータ推定、予測の良さを評価」を200回繰り返し、バイアスbの分布を描くことで、平均対数尤度より、最大対数尤度の方が平均的にどれぐらい大きいか(平均バイアス)、ということがわかる

Q. バイアス補正とは？

A.
* 平均的な$b$と最大対数尤度$\log L^*$の値を元に下記の計算式で平均対数尤度の推定量を得ること
* $E(\log L) = \log L^* - b$


Q. 最尤推定するパラメータを$k$個持つモデルの平均対数尤度の推定量を求める式は？


A.
* $E(\log L) = \log L^* - k$（解析的かつ一般的に導出されているらしい）
* 上記に$-2$をかけたものが$AIC$。
* $AIC = -2 \times (\log L^* -k)$
* 平均対数尤度は「統計モデルの予測の良さ」であり、それに $-2$をかけた$AIC$は「予測の悪さ」と解釈できる。AICが低いモデルを選べば良い。

### 4.5.3. ネストしているGLM間のAIC比較

* AICによるモデル選択とは、複数のモデルのAICを比較してより小さいAICのモデルを選ぶこと
* バイアスのばらつきは大きいため、AICで評価して良いか不安になる。しかし同一データを使ってネストしている複数モデルのバイアス補正量のさの分布はそれほどばらつきの大きいものにはならないため、AICは有用である。
* ではネストしていない複数のモデルで、AICによるモデル選択は可能だろうか。現時点では、「多分問題ないだろう」という理由で用いられている。

## 4.6. この章のまとめと参考文献

* 「当てはまりの良さ」最大対数尤度 $\log L^*$で「良い」モデルを選んではいけない。
* 「当てはまりの悪さ」逸脱度 $ D = -2 \log L^*$
* モデルは複雑化するだけで、最大対数尤度 $\log L^*$は改善されてしまう。そのためモデルの複雑さを考慮したAICでモデル選択しなければならない。
* AICは平均対数尤度の推定値。これは最大対数尤度 $\log L^*$のバイアス補正によって評価される。
* AICは当てはまりの良いモデルではなく、良い予測をするモデルを選ぶ基準。

### AICによるモデル選択における注意点
* モデル選択の目的は「真の」モデルを求めることではない
* データ数が少ない場合には、「真の」モデルより、単純なモデルの方が予測能力が高い可能性もある
* データ数が少ない場合には、選択されたパラメータ数が過大であっても、「余計な」パラメータの推定地はゼロに近づくので問題ない
* モデル選択は、データ解析者が用意したモデルたちの中から良いモデルを選んでいるので、よりAICが良いモデルが他に存在する可能性がある。

* その他のモデル選択基準としては、交差検証法、ブートストラップ情報量基準など。

# 表4.3の作成

In [16]:
model_list = ["y ~ 1","y ~ f","y ~ x","y ~ x + f"]
k_list = list()
logl_list = list()
deviance_2logl_list = list()
residual_deviance_list = list()
aic_list = list()

for model_str in model_list:
    model = smf.glm(model_str,data=data,family=sm.families.Poisson())
    result = model.fit()
    result.summary()
    
    k = result.df_model+1
    k_list.append(k)
    
    logl = result.llf
    logl_list.append(logl)

    deviance_2logl = - 2*logl
    deviance_2logl_list.append(deviance_2logl)
    
    residual_deviance = result.deviance
    residual_deviance_list.append(residual_deviance)



    aic = -2*(logl-k)
    aic_list.append(aic)

    
# fullmodelのデータを追加する
# 一部の数値は上で計算している
model_list.append("full model")
k_list.append(100)
logl_list.append(max_logl)
deviance_2logl_list.append(min_dev)
residual_deviance_list.append(0)
aic_list.append(-2*(max_logl-100))


In [17]:
df_aic = pd.DataFrame(index=model_list)
df_aic["k"] = k_list
df_aic["logl"] = logl_list
df_aic["deviance_2logl"] = deviance_2logl_list # 
df_aic["residual_deviance"] = residual_deviance_list #残差逸脱度
df_aic["aic"] = aic_list

In [18]:
df_aic

,k,logl,deviance_2logl,residual_deviance,aic
y ~ 1,1,-237.643221,475.286443,89.506938,477.286443
y ~ f,2,-237.627257,475.254514,89.475009,479.254514
y ~ x,2,-235.386251,470.772502,84.992996,474.772502
y ~ x + f,3,-235.293719,470.587438,84.807933,476.587438
full model,100,-192.889753,385.779505,0.000000,585.779505


* aicは低いものを選べば良いので、"y ~ x"のモデルを選ぶと良い

### 同様に、ポアソン回帰ではなく線形回帰で実施

In [19]:
model_list = ["y ~ 1","y ~ f","y ~ x","y ~ x + f"]
k_list = list()
logl_list = list()
deviance_2logl_list = list()
residual_deviance_list = list()
aic_list = list()

for model_str in model_list:
    # 線形回帰
    model = smf.glm(model_str,data=data)
    result = model.fit()
    result.summary()
    
    k = result.df_model+1
    k_list.append(k)
    
    logl = result.llf
    logl_list.append(logl)
    
    deviance_2logl = - 2*logl
    deviance_2logl_list.append(deviance_2logl)
    
    residual_deviance = result.deviance
    residual_deviance_list.append(residual_deviance)


    aic = -2*(logl-k)
    aic_list.append(aic)

    
df_aic = pd.DataFrame(index=model_list)
df_aic["k"] = k_list
df_aic["logl"] = logl_list
df_aic["deviance_2logl"] = deviance_2logl_list
df_aic["residual_deviance"] = residual_deviance_list
df_aic["aic"] = aic_list

df_aic

,k,logl,deviance_2logl,residual_deviance,aic
y ~ 1,1,-237.894891,475.789782,682.110000,477.789782
y ~ f,2,-237.876562,475.753124,681.860000,479.753124
y ~ x,2,-235.233125,470.466250,646.747281,474.466250
y ~ x + f,3,-235.118314,470.236628,645.263913,476.236628


* 上記の結果をどう判断すべきか
* AICでみると、『線形回帰の「y ~ x」』がもっとも低い。
* しかし、線形回帰は正規分布を前提とし、範囲が±∞となることから、定性的には正しくないように思える。
* このような時、どれを用いれば良いか。
* 学習結果を見てみると、「P>|z|」が0.05　以上となっていることが多かった。
    * P値？
    
* （勉強会で頂いたコメント）確率分布が違うとき、AICで比較してはいけないようだ。
    * [久保先生の資料](http://hosho.ees.hokudai.ac.jp/~kubo/ce/FaqModelSelection.html#toc14)によると、下記。
        * 離散分布 vs 連続分布の場合はダメ
            * 理由: 対数尤度の計算方法が離散分布と連続分布で異なるため …… それ以前の問題として，そもそもある観測データに対してこういう比較をすること自体がまったくナンセンスなんですけど

In [20]:
model = smf.glm("y ~ x",data=data)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                  100
Model:                            GLM   Df Residuals:                       98
Model Family:                Gaussian   Df Model:                            1
Link Function:               identity   Scale:                   6.59946204749
Method:                          IRLS   Log-Likelihood:                -235.23
Date:                Thu, 06 Jul 2017   Deviance:                       646.75
Time:                        00:48:26   Pearson chi2:                     647.
No. Iterations:                     2                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.8483      2.597      0.712      0.477      -3.241       6.938
x              0.5929      0.256      2.315      0.021       0.091       1.095
==============================================================================
"""

## 線形回帰のフルモデル

* 線形回帰の場合、確率はどのように求める？線形回帰の場合は連続値を取るため、ある値を取ることの確率が求められない。
* 何らかの形で範囲を決める必要？

# キーワードまとめ
| 名称| 説明 | 式 |
|:-----------:|:------------:|:------------:|
| 対数尤度       | 当てはまりの良さ| $\log L$ |
| 最大対数尤度 | もっとも当てはまりの良いパラメータで計算した対数尤度 | $\log L^*$ |
| 逸脱度 | 当てはまりの悪さ | $D = -2 \log L$ |
| 残差逸脱度 | 当てはまりの悪さの相対値 | $D-$(当該分布モデルで可能な最小逸脱度) |
| 平均対数尤度| 統計モデルの「予測」の良さ | $E(\log L)$ |
| バイアス| 最大対数尤度と平均対数尤度の差 | $b = \log L^* - E(\log L)$ |
| AIC| 統計モデルの予測の悪さ | $AIC = -2 \times (\log L^* - k) $|